In [1]:
import sys
sys.path.append("../")

In [2]:
import pandas as pd
import plotly.graph_objects as go
from technicals.indicators import RSI
from technicals.patterns import apply_patterns
from plotting import CandlePlot

In [3]:
df_raw = pd.read_pickle("../data/EUR_USD_H1.pkl")

In [4]:
df_raw.shape

(4000, 14)

In [5]:
df_an = df_raw.copy()#df_raw.iloc[-6000:].copy()
df_an.reset_index(drop=True, inplace=True)

In [6]:
df_an.shape

(4000, 14)

In [7]:
df_an = RSI(df_an)

In [8]:
df_an.tail()

,time,volume,mid_o,mid_h,mid_l,mid_c,bid_o,bid_h,bid_l,bid_c,ask_o,ask_h,ask_l,ask_c,RSI_14
3995,2023-04-02 21:00:00+00:00,211,1.08416,1.08443,1.08404,1.08415,1.08391,1.08427,1.08361,1.08372,1.08440,1.08470,1.08420,1.08458,33.941409
3996,2023-04-02 22:00:00+00:00,3892,1.08415,1.08425,1.08052,1.08196,1.08370,1.08399,1.08042,1.08186,1.08460,1.08475,1.08062,1.08205,27.678999
3997,2023-04-02 23:00:00+00:00,2906,1.08195,1.08231,1.08012,1.08026,1.08185,1.08224,1.08004,1.08018,1.08205,1.08239,1.08020,1.08035,23.980250
3998,2023-04-03 00:00:00+00:00,3934,1.08027,1.08148,1.07987,1.08139,1.08017,1.08139,1.07979,1.08132,1.08037,1.08156,1.07995,1.08146,30.617227
3999,2023-04-03 01:00:00+00:00,4183,1.08138,1.08170,1.08028,1.08076,1.08132,1.08162,1.08021,1.08069,1.08145,1.08177,1.08036,1.08084,29.092234


In [9]:
df_an = apply_patterns(df_an)

In [10]:
df_an['EMA_200'] = df_an.mid_c.ewm(span=200, min_periods=200).mean()

In [11]:
df_an.columns

Index(['time', 'volume', 'mid_o', 'mid_h', 'mid_l', 'mid_c', 'bid_o', 'bid_h',
       'bid_l', 'bid_c', 'ask_o', 'ask_h', 'ask_l', 'ask_c', 'RSI_14',
       'body_lower', 'body_upper', 'body_bottom_perc', 'body_top_perc',
       'body_perc', 'direction', 'body_size', 'low_change', 'high_change',
       'body_size_change', 'mid_point', 'mid_point_prev_2', 'body_size_prev',
       'direction_prev', 'direction_prev_2', 'body_perc_prev',
       'body_perc_prev_2', 'HANGING_MAN', 'SHOOTING_STAR', 'SPINNING_TOP',
       'MARUBOZU', 'ENGULFING', 'TWEEZER_TOP', 'TWEEZER_BOTTOM',
       'MORNING_STAR', 'EVENING_STAR', 'EMA_200'],
      dtype='object')

In [12]:
our_cols = ['time', 'mid_o', 'mid_h', 'mid_l', 'mid_c',
                'ask_c','bid_c', 'ENGULFING', 'direction', 'EMA_200', 'RSI_14' ]

In [13]:
df_slim = df_an[our_cols].copy()
df_slim.dropna(inplace=True)
df_slim.reset_index(drop=True, inplace=True)

In [14]:
df_slim.head()

,time,mid_o,mid_h,mid_l,mid_c,ask_c,bid_c,ENGULFING,direction,EMA_200,RSI_14
0,2022-08-22 14:00:00+00:00,0.99696,0.99772,0.99602,0.99668,0.99675,0.99662,False,-1,1.013535,23.866165
1,2022-08-22 15:00:00+00:00,0.99670,0.99678,0.99276,0.99348,0.99355,0.99341,False,-1,1.013304,18.650639
2,2022-08-22 16:00:00+00:00,0.99349,0.99494,0.99302,0.99320,0.99327,0.99312,False,-1,1.013074,18.274326
3,2022-08-22 17:00:00+00:00,0.99321,0.99379,0.99262,0.99276,0.99283,0.99268,False,-1,1.012841,17.670941
4,2022-08-22 18:00:00+00:00,0.99276,0.99368,0.99269,0.99332,0.99339,0.99325,True,1,1.012618,21.235485


### start test the strategy

In [15]:
BUY = 1
SELL = -1
NONE = 0
RSI_LIMIT = 50.0

def apply_signal(row):
    if row.ENGULFING == True:
        if row.direction == BUY and row.mid_l > row.EMA_200:
            if row.RSI_14 > RSI_LIMIT:
                return BUY
        if row.direction == SELL and row.mid_h < row.EMA_200:
            if row.RSI_14 < RSI_LIMIT:
                return SELL
    return NONE        

In [16]:
df_slim["SIGNAL"] = df_slim.apply(apply_signal, axis=1)

In [17]:
df_slim["SIGNAL"].value_counts()

 0    3447
 1     223
-1     131
Name: SIGNAL, dtype: int64

In [18]:
LOSS_FACTOR = -1.0
PROFIT_FACTOR = 1.5

def apply_take_profit(row):
    if row.SIGNAL != NONE:
        return (row.mid_c - row.mid_o) * 1.5 + row.mid_c
    else:
        return 0.0

def apply_stop_loss(row):
    if row.SIGNAL != NONE:
        return row.mid_o
    else:
        return 0.0

In [19]:
df_slim["TP"] = df_slim.apply(apply_take_profit, axis=1)
df_slim["SL"] = df_slim.apply(apply_stop_loss, axis=1)

In [20]:
df_slim[df_slim.SIGNAL==BUY].head()

,time,mid_o,mid_h,mid_l,mid_c,ask_c,bid_c,ENGULFING,direction,EMA_200,RSI_14,SIGNAL,TP,SL
137,2022-08-30 09:00:00+00:00,1.00246,1.00550,1.00240,1.00327,1.00334,1.00320,True,1,1.001137,63.773543,1,1.004485,1.00246
291,2022-09-07 19:00:00+00:00,0.99820,1.00100,0.99816,1.00086,1.00093,1.00080,True,1,0.996253,69.802522,1,1.004850,0.99820
298,2022-09-08 02:00:00+00:00,0.99847,0.99903,0.99843,0.99875,0.99882,0.99868,True,1,0.996485,60.898612,1,0.999170,0.99847
300,2022-09-08 04:00:00+00:00,0.99864,0.99947,0.99862,0.99899,0.99907,0.99891,True,1,0.996531,61.444528,1,0.999515,0.99864
315,2022-09-08 19:00:00+00:00,0.99914,0.99992,0.99912,0.99992,0.99999,0.99984,True,1,0.996858,56.928414,1,1.001090,0.99914


In [29]:
df_plot = df_slim.iloc[70:100]
cp = CandlePlot(df_plot, candles=True)

trades = cp.df_plot[cp.df_plot.SIGNAL != NONE]

markers = ['mid_c', 'TP', 'SL']
marker_colors = ['#0000FF', '#00FF00', '#FF0000']

for i in range(3):
    cp.fig.add_trace(go.Scatter(
        x = trades.sTime,
        y = trades[markers[i]],
        mode = 'markers',
        marker=dict(color=marker_colors[i], size=12)
    ))

cp.show_plot(line_traces=["EMA_200"], sec_traces=['RSI_14'], height=250)

### check the gain of strategy

In [22]:
class Trade:
    def __init__(self, row):
        self.running = True
        self.start_index = row.name
        self.start_price = row.mid_c
        self.trigger_price = row.mid_c
        self.SIGNAL = row.SIGNAL
        self.TP = row.TP
        self.SL = row.SL
        self.result = 0.0
        self.end_time = row.time
        self.start_time = row.time
        self.duration = 0
        
    def close_trade(self, row, result, trigger_price):
        self.running = False
        self.result = result
        self.end_time = row.time
        self.trigger_price = trigger_price
        
    def update(self, row):
        self.duration += 1
        if self.SIGNAL == BUY:
            if row.mid_h >= self.TP:
                self.close_trade(row, PROFIT_FACTOR, row.mid_h)
            elif row.mid_l <= self.SL:
                self.close_trade(row, LOSS_FACTOR, row.mid_l)
        if self.SIGNAL == SELL:
            if row.mid_l <= self.TP:
                self.close_trade(row, PROFIT_FACTOR, row.mid_l)
            elif row.mid_h >= self.SL:
                self.close_trade(row, LOSS_FACTOR, row.mid_h)    

In [23]:
open_trades = []
closed_trades = []

for index, row in df_slim.iterrows(): # iterrows is slow, can use items[index]
    for ot in open_trades:
        ot.update(row)
        if ot.running == False:
            closed_trades.append(ot)
    open_trades = [x for x in open_trades if x.running == True]
    
    if row.SIGNAL != NONE:
        open_trades.append(Trade(row))    

In [24]:
df_results = pd.DataFrame.from_dict([vars(x) for x in closed_trades])

In [25]:
df_results.result.sum()

23.5

### inspect results

In [26]:
df_results.sort_values(by="start_index", inplace=True)

In [27]:
df_results.head()

,running,start_index,start_price,trigger_price,SIGNAL,TP,SL,result,end_time,start_time,duration
0,False,39,0.99428,0.99594,-1,0.99209,0.99574,-1.0,2022-08-24 07:00:00+00:00,2022-08-24 05:00:00+00:00,2
1,False,42,0.99462,0.99585,-1,0.99354,0.99534,-1.0,2022-08-24 09:00:00+00:00,2022-08-24 08:00:00+00:00,1
2,False,44,0.99218,0.99948,-1,0.98801,0.99496,-1.0,2022-08-24 14:00:00+00:00,2022-08-24 10:00:00+00:00,4
3,False,47,0.99178,0.99948,-1,0.99019,0.99284,-1.0,2022-08-24 14:00:00+00:00,2022-08-24 13:00:00+00:00,1
5,False,73,0.99570,0.99912,-1,0.99210,0.99810,-1.0,2022-08-26 08:00:00+00:00,2022-08-25 15:00:00+00:00,17


### sync M5 and H1 candles

In [30]:
df_m5 = pd.read_pickle("../data/EUR_USD_M5.pkl")

FileNotFoundError: [Errno 2] No such file or directory: '../data/EUR_USD_M5.pkl'

In [ ]:
df_m5.shape

In [ ]:
df_m5.time.max()

In [ ]:
df_raw.time.max()

In [ ]:
from dateutil import parser

In [ ]:
time_min = parser.parse("2021-12-15T10:00:00Z")
time_max = parser.parse("2021-12-15T11:00:00Z")
df_m5_s = df_m5[(df_m5.time>=time_min)&(df_m5.time<=time_max)]
df_raw_s = df_raw[(df_raw.time>=time_min)&(df_raw.time<=time_max)]

In [ ]:
df_m5_s

In [ ]:
df_raw_s